# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [40]:
# Write your code below.
import os
import time
# set working dir to same as env file
os.chdir('../../01_materials/labs/')


%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [41]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [42]:
import os
from glob import glob

# Write your code below.
ft_dir = os.getenv("PRICE_DATA")
parq_file = glob(os.path.join(ft_dir, "*/*/*.parquet"), recursive=True)



In [43]:
parq_file

['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015.pa

In [44]:
df = dd.read_parquet(parq_file).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [45]:
# Importing the necessary libraries

import numpy as np

dd_feat = (df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))


C:\Users\klawa\AppData\Local\Temp\ipykernel_9956\1595277154.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (df.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [46]:
# Loading the necessary packages
import pandas as pd

# converting to pandas dataframe
dd_feat = dd_feat.compute()

# Calculate rolling average with a window of 10 days

pd_feat1 = dd_feat.groupby('ticker', group_keys=False).apply (
    lambda x: x.assign(rolling_avg_rtn=x['returns'].rolling(10).mean())
)
pd_feat1


Please comment: 
The operation of calculating the moving average can be completed within dask, and we dont need to read in pandas. It would be better to keep the workflow within dask so as to leverage its capabilities to read large data set within a short space of time compared to reading it to a dataframe that can be time consuming. 

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.